In [1]:
!pip install nltk


In [2]:
!pip install pandas

In [29]:
import pandas as pd
import nltk
from nltk.corpus import twitter_samples
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.classify import NaiveBayesClassifier
nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
import nltk.classify.util


df=pd.read_csv('youtoxic_english_1000(2).csv',usecols=[2,15])
# podzial na texty i ich kasyfikacje
i=0
banedComments=[]
notBanedComments=[]
textsArray=df.drop("IsBaned",axis=1).to_numpy()
IsBanedArray=df["IsBaned"].to_numpy()

# funkcja do tworzenia arrayow z labelem true i zlabelem false
def add_to_array(text,index):
  # print(textsArray[index][0])
  if(text==True):
    banedComments.append(textsArray[index][0])
  else:
    notBanedComments.append(textsArray[index][0])
# tworzenie arrayow
for i in range(len(IsBanedArray)):
  add_to_array(IsBanedArray[i],i)

# robiwnie labelow i ich tasowanie
comments=banedComments+notBanedComments
labels = ['True'] * len(banedComments) + ['False'] * len(notBanedComments)
# comments
# Shuffle the dataset
combined = list(zip(comments, labels))
random.shuffle(combined)
comments, labels = zip(*combined)
# array to text i tokenizacja
sample_text = ','.join(comments)
tokens = word_tokenize(sample_text)

# usuwanie tokenów
stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]
filtered_tokens = remove_stopwords(tokens)
# stematyzacja i lematyzacja tokenow
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()

# stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
# lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

# robienie bag of words
all_words = [word.lower() for comment in comments for word in word_tokenize(comment)]
all_words_freq = FreqDist(all_words)

# Select the top 10000 words as features
word_features = list(all_words_freq.keys())[:10000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

# Create feature sets for training and testing
feature_sets = [(document_features(word_tokenize(comment)), label) for (comment, label) in zip(comments, labels)]
train_set, test_set = feature_sets[300:], feature_sets[:300]
# tranowanie modela
classifier = NaiveBayesClassifier.train(train_set)

accuracy = nltk.classify.util.accuracy(classifier, test_set)
print(f'Accuracy: {accuracy * 100:.2f}%')

# //////////////////////////////////////////
# function to get precision recall and accuracy
def classiffy_train_set(sentence):
  # test_features = document_features(word_tokenize(sentence))
  # print(test_features)
  classification = classifier.classify(sentence)
  return classification


# ocena modela
predictions=[]
number=300
TP=0
TN=0
FP=0
FN=0
for i in range(number):

   predictions.append(classiffy_train_set(test_set[i][0]))
# print(IsBanedArray[0])
for i in range(number):
  if(str(test_set[i][1])=="True"):
    if(str(predictions[i])==str(test_set[i][1])):
      TP=TP+1
    else:
      FN=FN+1
  else:
    if(str(predictions[i])==str(test_set[i][1])):
      TN=TN+1
    else:
      FP=FP+1
print ("true positive=  "+str(TP))
print ("true negative=  "+str(TN))
print ("false positive=  "+str(FP))
print ("false negative=  "+str(FN))
precision=TP/(TP+FP)
recall=TP/(TP+FN)
accuracy=(TP+TN)/number
print("precision= "+str(precision))
print("recall= "+str(recall))
print("accuracy= "+str(accuracy))

# definicja nowego textu(nie w training set)
def classiffy_sentence(sentence):
  test_features = document_features(word_tokenize(sentence))
  # print(test_features)
  classification = classifier.classify(test_features)
  return classification



[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Accuracy: 65.33%
true positive=  73
true negative=  123
false positive=  35
false negative=  69
precision= 0.6759259259259259
recall= 0.5140845070422535
accuracy= 0.6533333333333333


In [30]:
print(classiffy_sentence("This comment seems offensive and should be banned"))


False
